### Cách đọc file thông thường và tf.data

#### ImageDataGenerator

In [1]:
%cd "/content/drive/MyDrive/grocery-images"

/content/drive/.shortcut-targets-by-id/1gVocdgOjp7dnVo8rM6lz1Io7VjaCD6nR/grocery-images


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.data import AUTOTUNE
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras import models, layers
from imutils import paths
import tensorflow as tf 
import numpy as np
import pandas as pd
import os
import time

In [3]:
def benchmark(datasetGen, numSteps):
  start = time.time()
  
  for i in range(0, numSteps):
    images, labels = next(datasetGen)

  end = time.time()
  return end - start

In [4]:
cur_dir = os.getcwd()

In [103]:
numSteps = 1
BS = 64

In [101]:
imageGen = ImageDataGenerator(rescale=1/.255, 
                              preprocessing_function=mobilenet_v2.preprocess_input)
dataGen = imageGen.flow_from_directory(
    cur_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=BS
)

Found 623 images belonging to 66 classes.


In [19]:
numSteps = 1
BS = 64

In [20]:
totalTime = benchmark(dataGen, numSteps)
print(f'ImageDataGenerator generates {BS * numSteps} images in {totalTime}')

1
ImageDataGenerator generates 64 images in 83.21854591369629


Dữ liệu chỉ 623 ảnh và batch size khá lớn nhưng phải tốn 18s để load dữ liệu một lần -> không hiệu quả

#### tf.data

In [127]:
def load_images(imagePath, pretrained_net, required_size=(160, 160)):
  image = tf.io.read_file(imagePath) # đọc file
  image = tf.cast(tf.image.decode_png(image, channels=3), tf.float32) / 255.0 # chuyển về ảnh png
  image = tf.image.resize(image, required_size) # kích thước đầu vào
  image = pretrained_net.preprocess_input(image) # tiền xử lí theo pretrained model

  label = tf.strings.split(imagePath, os.path.sep)[-2] # lấy tên nhãn
  oneHot = tf.cast(label == classNames, tf.int32) # thực hiện one-hot encoding
  encodedLabel = tf.argmax(oneHot) # lấy giá trị lớn nhất - vị trí của nhãn
  return image, encodedLabel

In [11]:
imagePath = list(paths.list_images(cur_dir))
classNames = sorted(os.listdir(cur_dir))

In [128]:
def create_dataset(BS, pretrained_net):
  dataset = tf.data.Dataset.from_tensor_slices(imagePath)
  dataset = (dataset
            .shuffle(1024) # (1)
            .map(lambda x: load_images(x, pretrained_net), num_parallel_calls=AUTOTUNE) # (2)
            .cache() # (3)
            .repeat() # (4)
            .batch(BS) # (5)
            .prefetch(AUTOTUNE) # (6) 
            )
  return dataset

(1): khởi tạo một buffer có kích thước truyền vào, mỗi step lấy ra từ buffer số lượng dữ liệu tương ứng 1 batch size và lấy dữ liệu từ tập nguồn để làm đầy lại

(2): vì dữ liệu truyền vào là một danh sách các đường dẫn ảnh => hàm map ánh xạ từng phần tử và thực hiện phép biến đổi tương ứng

(3): caching để ghi nhớ lại dữ liệu đã sử dụng, tăng tốc độ tính toán

(4): Khi lấy hết dữ liệu từ tập nguồn thì cần làm đầy lại

(5): số lượng ảnh mỗi batch

(6): thực hiện song song việc chuẩn bị dữ liệu trước đó và tính toán dữ liệu hiện tại

In [17]:
dataset = create_dataset(BS, mobilenet_v2)
totalTime = benchmark(iter(dataset), numSteps)
print(f'tf.data API generates {BS * numSteps} in {totalTime}')

1
tf.data API generates 64 in 44.91099834442139


tf.data API nhanh hơn gấp 2 lần ImageDataGenerator cho mỗi bước(dữ liệu trong 1 batch), con số này càng dao động đáng kể khi dữ liệu càng lớn

=> Sử dụng tf.data

### Baseline

In [143]:
mobilenet = MobileNetV2(include_top=False, weights='imagenet', input_shape=(160, 160, 3), pooling='avg')

thông thường batch size lớn sẽ lợi về thời gian tuy nhiên sẽ mất đi chút chính xác

dữ liệu tương đối ít nên sử dụng batch size nhỏ


In [138]:
BS = 16
epochs = 15
num_images = 623
steps_per_epoch = num_images // BS

In [139]:
dataset = create_dataset(BS, mobilenet_v2)

In [145]:
mobilenet.trainable = False

model = models.Sequential([
    mobilenet,
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(classNames.__len__(), activation='softmax')
]
)
model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'])

In [146]:
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/15
38/38 [==============================] - 87s 2s/step - loss: 4.2287 - acc: 0.0115
Epoch 2/15
38/38 [==============================] - 3s 31ms/step - loss: 4.1925 - acc: 0.0115
Epoch 3/15
38/38 [==============================] - 1s 27ms/step - loss: 4.1886 - acc: 0.0247
Epoch 4/15
38/38 [==============================] - 1s 28ms/step - loss: 4.1878 - acc: 0.0230
Epoch 5/15
38/38 [==============================] - 1s 28ms/step - loss: 4.1866 - acc: 0.0247
Epoch 6/15
38/38 [==============================] - 1s 29ms/step - loss: 4.1850 - acc: 0.0247
Epoch 7/15
38/38 [==============================] - 1s 28ms/step - loss: 4.1843 - acc: 0.0230
Epoch 8/15
38/38 [==============================] - 1s 28ms/step - loss: 4.1830 - acc: 0.0247
Epoch 9/15
13/38 [=========>....................] - ETA: 0s - loss: 4.1892 - acc: 0.0144

KeyboardInterrupt: ignored